# Чтение данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
import json

In [3]:
import warnings
import warnings
warnings.filterwarnings('ignore', category=Warning)

In [4]:
tags = pd.read_csv('data/tags.csv', sep=',#', names=['tag', 'name', 'type'])
tags = tags.loc[tags['type'] == '"1"']
tags.head(10)

,tag,name,type
151,"""179""","""О войне""","""1"""
152,"""180""","""военный""","""1"""
153,"""181""","""заграничная поездка""","""1"""
154,"""182""","""блокада""","""1"""
155,"""183""","""тыл""","""1"""
156,"""184""","""фронтовой""","""1"""
157,"""185""","""депортация""","""1"""
158,"""186""","""детский""","""1"""
159,"""187""","""профессиональный""","""1"""
160,"""188""","""путешествие""","""1"""


Всего размечено 86 тем. Из таблицы можно заметить, что есть темы, которые пересекаются, например "О войне", "военный", "фронтовый", "тыл".

In [5]:
tags.shape

(86, 3)

In [5]:
# список тем типа 1
themes = list(tags['tag'])

In [6]:
diaries = pd.read_csv('data/diary.csv', sep=',#', header=None)
diaries.head()

,0,1,2,3,4,5,6,7,8,9,10
0,"""3""","""4""","""published""","""329""","""1932-09-28""","""1937-04-30""",NaN,"""0""","""0""","""0""","""1425446154"""
1,"""4""","""5""","""published""","""356""","""1942-09-09""","""1944-01-27""",NaN,"""0""","""0""","""1474033861""","""1425446228"""
2,"""5""","""6""","""published""","""28""","""1943-01-31""","""1951-07-03""",NaN,"""0""","""0""","""0""","""1425446370"""
3,"""6""","""7""","""published""","""65""","""0000-00-00""","""0000-00-00""",NaN,"""0""","""0""","""0""","""1425446645"""
4,"""8""","""11""","""published""","""804""","""1943-08-04""","""1961-12-31""",NaN,"""0""","""0""","""0""","""1425487703"""


Всего есть 2342 дневника.

In [8]:
len(diaries[0])

2342

In [7]:
tags_diaries = pd.read_csv('data/tags_diaries.csv', sep=',#', names=['diary', 'tag'])
tags_diaries.head()

,diary,tag
0,"""3""","""181"""
1,"""3""","""328"""
2,"""4""","""180"""
3,"""4""","""184"""
4,"""8""","""187"""


Из них хоть какой-то тэг приписан к 1407 дневникам. К одному дневнику может быть приписано несколько тэгов.

In [8]:
len(set(tags_diaries['diary']))

1407

In [9]:
# датафрейм с id дневников, которым приписана хотя бы одна тема
diaries_with_themes = tags_diaries[tags_diaries['tag'].isin(themes)]

Тема приписана 1407 дневникам. Многим дневникам приписано несколько тем. Видимо, потому, что некоторое темы пересекаются или у одного дневника может быть несколько разных тем. 

Как можно заметить, локации и персоны ни одному дневнику не приписаны. 

In [10]:
len(set(diaries_with_themes['diary']))

1407

In [11]:
with open('data/notes.json', 'r') as f:
        notes = json.load(f)

В джейсоне 182975 записи (кажется, в csv было больше).

In [15]:
len(notes[2]['data'])

182975

In [26]:
notes[2]['data'][-1]

{'createdDate': '1485536357',
 'date': '2002-03-18',
 'dateTop': '0000-00-00',
 'diary': '145',
 'id': '221877',
 'julian_calendar': '0',
 'notDated': '0',
 'pictures': '1',
 'text': 'Впервые в жизни получил какое-то удовольствие от тренажера. Нынче он у нас стал компьютерным, цветным, адекватным. Не все, конечно, еще отлажено, но против старого — небо и земля. На нем можно решать задачи. С пожаром на взлете мы приземлились за 2.45, визуально, ну, с чуть взмокшей спиной. Так это ж я, старик; молодые-то быстрее зайдут.    \nПриехала газетчица Маша; дождался, посмотрел материал. От всего разговора, в статью вошли, конечно, крохи, и не самые лучшие; пришлось править, кое-что даже принципиально. Для газетенки сойдет. Сказала, читайте послезавтра в газете… Кофейком ее напоил… но все как-то на скоростях. Оно и понятно: репортеры — они как хариусы: хвать на лету — и дальше…    \nЧем мне, собственно, не понравился материал? Тем, что в этом интервью о моей большой книге — ни слова. Вообще, на м

In [12]:
rows = []

for note in notes[2]['data']:
    rows.append([note['text'], note['diary']])

In [13]:
notes = pd.DataFrame(rows, columns=['text', 'diary'])

In [14]:
# датафрейм с записями и id их дневника
notes.head()

,text,diary
0,Барабинск. Татарка.,130
1,Омск. Утро. Стоим до вечера. Обед. Сухой паек.,130
2,В 6 часов утра еще темно. Станция Называевская...,130
3,Утром рано приехали в Тюкалинск. Из автоотряда...,130
4,"Так давно я не прикасался к дневнику, что он м...",234


# Предобработка

In [15]:
from stop_words import get_stop_words
import nltk
from pymystem3 import Mystem
import re

In [16]:
random_seed = 42

In [17]:
stop_words = get_stop_words('ru')
stop_words.append('свой')

In [22]:
m = Mystem()

In [23]:
def tokenize(text):
    comp = re.compile('[А-ЯЁа-яё\-]+')
    tokens = comp.findall(text.lower())
    return tokens

In [24]:
def lemmatize(text):
    return [m.lemmatize(word)[0] for word in text]

In [25]:
# токенизируем заметки
notes_lemma = notes.copy()
notes_lemma['text'] = notes.text.apply(lambda x: tokenize(x))

# Тематическое моделирование

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import FeatureUnion

In [19]:
tfidf = TfidfVectorizer(sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stop_words,
    ngram_range=(1, 2))

In [20]:
notes_vec = tfidf.fit_transform(notes.text)

In [21]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [22]:
tfidf_feature_names = tfidf.get_feature_names()

In [26]:
no_topics = 5

# NMF
nmf = NMF(n_components=no_topics, random_state=random_seed, alpha=.1, l1_ratio=.5, init='nndsvd').fit(notes_vec)

MemoryError: 

In [27]:
# LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50., 
                                random_state=random_seed).fit(notes_vec)

/home/socur/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


MemoryError: 